#HealChat

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.2 MB/s eta 0:00:00


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
#run to clear out history
history = []

In [4]:
def join_list(list1):
  kalimat = " ".join(list1)
  return kalimat

Integrating with Gemini

In [5]:
# Used to securely store your API key
from google.colab import userdata

In [6]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
model_gemini = genai.GenerativeModel('gemini-1.5-pro')

Integrating Emotion Classification & Gemini

In [8]:
#Run to predict user input
text = input('Ceritakan Harimu:')
history.append(text)

Ceritakan Harimu:hi, aku nangis banget, huhuhu


In [9]:
#Run to generate response
prompt = "" #input specific prompt
response = model_gemini.generate_content(f"Anggap dia temanmu yang sedang bercerita. Dia bercerita/membalas kalau {text}. Respon dia sebagai teman. Jangan gunakan bahasa yang terlalu kaku dan jangan terlalu panjang. List berikut adalah chat dia dengan mu {history}")
to_markdown(response.text)

> Eh kenapa nih? Cerita dong sama aku 😭 Ada yang bisa kubantu? 


In [10]:
history

['hi, aku nangis banget, huhuhu']

In [11]:
join_list(history)

'hi, aku nangis banget, huhuhu'

#Check

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

In [13]:
#Load model
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Muat model dan tokenizer dari Google Drive
model_path = '/content/drive/My Drive/Github/Fix_Github/Emotion Classification/model'
tokenizer_path = '/content/drive/My Drive/Github/Fix_Github/Emotion Classification/tokenizer'

from transformers import pipeline

# Gunakan model dan tokenizer yang dimuat dari Google Drive
nlp = pipeline(
    "sentiment-analysis",
    model=model_path,
    tokenizer=tokenizer_path,
    return_all_scores=True
)

Some layers from the model checkpoint at /content/drive/My Drive/Github/Fix_Github/Emotion Classification/model were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/My Drive/Github/Fix_Github/Emotion Classification/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predict

In [15]:
label_mapping = {'Marah': 'marah', 'Takut': 'takut', 'Senang': 'senang', 'Cinta': 'cinta', 'Sedih': 'sedih'}

In [16]:
# Fungsi untuk menyimpan probabilitas label
def predict_text_probabilities(hasil_nlp,text):
    mapped_probabilities = {}
    for item in hasil_nlp[0]:  # Assuming we only need the first set of predictions
        label = item['label']
        score = item['score']
        mapped_probabilities[label_mapping[label]] = score

    return mapped_probabilities

In [17]:
text = join_list(history)
probabilities = predict_text_probabilities(nlp(text),text)
for label, prob in probabilities.items():
    print(f"{label}: {prob:.4f}")

max_emotion = max(probabilities, key=probabilities.get)
max_emotion_prob = probabilities[max_emotion]
print(f"\nEmosi pengguna: {max_emotion} ({max_emotion_prob:.4f})")

marah: 0.0045
takut: 0.0097
senang: 0.0710
cinta: 0.1378
sedih: 0.7771

Emosi pengguna: sedih (0.7771)


**Activity A**

In [23]:
#Run to generate response
prompt = "" #input specific prompt
response = model_gemini.generate_content(f"User kita sedang dalam keadaan {max_emotion} dengan probabilisitas {max_emotion_prob}.  Berikan 3 rekomendasi aktivitas untuknya tanpa kata kata penyemangat output hanya 3 aktivitas saja")
to_markdown(response.text)

> 1. Mendengarkan musik instrumental yang menenangkan.
> 2. Berjalan santai di alam terbuka.
> 3. Mandi air hangat dengan aroma terapi. 


**Activity B**



In [19]:
import random
import json

In [20]:
recommended_activities = {
    "sedih": [
        {"title": "Olahraga", "description": "Pergi berjalan, berlari, atau lakukan latihan di rumah."},
        {"title": "Mendengarkan Musik", "description": "Dengarkan musik yang membangkitkan semangat atau menenangkan."},
        {"title": "Bicara dengan Seseorang", "description": "Bagikan perasaan Anda dengan teman atau anggota keluarga."},
        {"title": "Terlibat dalam Hobi", "description": "Lakukan sesuatu yang Anda nikmati, seperti membaca, melukis, atau berkebun."},
        {"title": "Menulis di Jurnal", "description": "Ungkapkan pikiran dan perasaan Anda di atas kertas."}
    ],
    "senang": [
        {"title": "Merayakan", "description": "Bagikan kebahagiaan Anda dengan teman dan keluarga."},
        {"title": "Olahraga", "description": "Salurkan energi positif Anda ke dalam aktivitas fisik."},
        {"title": "Berkegiatan Sosial", "description": "Gunakan suasana hati positif Anda untuk membantu orang lain."},
        {"title": "Buat Sesuatu", "description": "Salurkan kebahagiaan Anda ke dalam aktivitas kreatif seperti menggambar, memasak, atau membuat kerajinan."},
        {"title": "Menghabiskan Waktu di Luar Ruangan", "description": "Nikmati berjalan di alam, pergi ke pantai, atau berpiknik."}
    ],
    "takut": [
        {"title": "Latihan Pernapasan Dalam", "description": "Lakukan latihan pernapasan dalam untuk menenangkan pikiran Anda."},
        {"title": "Meditasi Kesadaran", "description": "Fokus pada momen saat ini untuk mengurangi kecemasan."},
        {"title": "Hadapi Ketakutan Anda", "description": "Secara bertahap hadapi sumber ketakutan Anda dalam langkah-langkah kecil yang dapat diatasi."},
        {"title": "Bicarakan", "description": "Diskusikan ketakutan Anda dengan teman terpercaya atau terapis."},
        {"title": "Aktivitas Fisik", "description": "Lakukan aktivitas seperti yoga, tai chi, atau berjalan cepat untuk mengurangi stres."}
    ],
    "cinta": [
        {"title": "Menghabiskan Waktu Berkualitas", "description": "Rencanakan kencan atau aktivitas spesial dengan orang yang Anda cintai."},
        {"title": "Ekspresikan Rasa Syukur", "description": "Tulis surat atau catatan untuk mengungkapkan penghargaan."},
        {"title": "Lakukan Sesuatu yang Baik", "description": "Lakukan tindakan kebaikan untuk seseorang yang Anda pedulikan."},
        {"title": "Masak Bersama", "description": "Nikmati memasak dan berbagi makanan bersama orang tercinta."},
        {"title": "Terlibat dalam Kegiatan Bersama", "description": "Ikuti hobi atau aktivitas yang Anda berdua nikmati."}
    ],
    "marah": [
        {"title": "Aktivitas Fisik", "description": "Lepaskan energi terpendam melalui olahraga atau permainan."},
        {"title": "Pernapasan Dalam", "description": "Lakukan latihan pernapasan dalam atau relaksasi otot progresif."},
        {"title": "Saluran Kreatif", "description": "Salurkan kemarahan Anda ke dalam seni, musik, atau menulis."},
        {"title": "Waktu Sendiri", "description": "Ambil jeda dari situasi untuk menenangkan diri."},
        {"title": "Bicarakan", "description": "Diskusikan perasaan Anda dengan seseorang yang Anda percaya untuk mendapatkan perspektif."}
    ]
}

# Save the dictionary to a JSON file
with open('/content/recommended_activities.json', 'w') as json_file:
    json.dump(recommended_activities, json_file, indent=4)

In [21]:
# Load the JSON file
with open('/content/recommended_activities.json', 'r') as json_file:
    recommended_activities = json.load(json_file)

In [22]:
if max_emotion in recommended_activities:
    selected_activities = random.sample(recommended_activities[max_emotion], 3)
    num = 1
    # Format selected activities in Markdown
    print(f"Recommendations for {max_emotion}:\n")
    for activity in selected_activities:

      print(f"{num}. {activity['title']}\n{activity['description']}\n")
      num += 1
else:
    print("No recommendations available for the given emotion.")

Recommendations for sedih:

1. Menulis di Jurnal
Ungkapkan pikiran dan perasaan Anda di atas kertas.

2. Terlibat dalam Hobi
Lakukan sesuatu yang Anda nikmati, seperti membaca, melukis, atau berkebun.

3. Olahraga
Pergi berjalan, berlari, atau lakukan latihan di rumah.

